In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
def get_sales_url(base_url):
    sales_url = base_url.replace("UseSearch=no", "mode=sales&UseSearch=no")
    sales_url+="&jur=041&taxyr=2022&LMparent=20"
    
    return sales_url

def get_resident_url(base_url):
    resident_url = base_url.replace("UseSearch=no", "mode=residential&UseSearch=no")
    resident_url+="&jur=041&taxyr=2022&LMparent=20"
    
    return resident_url

In [4]:
import re

def process_text(item):
    if not item:
        return ""
    
    text=""
    for i in item.children:
        text+=i.text
        text+="\n"
    return(text)
        

def safe_search(regex, text):
    try:
        return re.search(regex, text).group(0)
    except:
        return None

def extract_resident_info(resident_url):
    page = requests.get(resident_url)
    soup = BeautifulSoup(page.content, "html.parser")
    text = process_text(soup.find(id="Residential"))
    
    year_built = safe_search(r'(?<=Year Built)[^.\s]*',text)
    bedrooms = safe_search(r'(?<=Bedrooms)[^.\s]*',text)
    square_feet = safe_search(r'(?<=Bedrooms)[^.\s]*',text)
    full_baths = safe_search(r'(?<=Full Baths)[^.\s]*',text)
    half_baths = safe_search(r'(?<=Half Baths)[^.\s]*',text)
    
    return { \
            "Year Built": year_built, \
            "Bedrooms": bedrooms, \
            "Square Feet": square_feet, \
            "Full Baths": full_baths, \
            "Half Baths": half_baths \
           }

def extract_sales_info(sales_url):
    page = requests.get(sales_url)
    soup = BeautifulSoup(page.content, "html.parser")
    text = process_text(soup.find(id="Sales"))
    
    try:
        sales = [(i.split("$")[0], i.split("$")[1].replace(",","")) for i in re.findall("\d+-\d+-\d+\$[0-9,]+", text)]
    except:
        sales = []
    
    return {"Sales": sales}

def extract_parcel(base_url):
    page = requests.get(base_url)
    soup = BeautifulSoup(page.content, "html.parser")
    text = process_text(soup.find(id="Parcel"))
    
    property_location = safe_search(r'(?<=Property Location).*',text)
    _class = safe_search(r'(?<=Class).*',text)
    land_use_code = safe_search(r'(?<=Land Use Code).*',text)
    municipality = safe_search(r'(?<=Municipality).*',text)
    
    return { \
            "Property Location": property_location, \
            "Class": _class, \
            "Land Use Code": land_use_code, \
            "Municipality": municipality \
           }

def extract_utilities(base_url):
    page = requests.get(base_url)
    soup = BeautifulSoup(page.content, "html.parser")
    text = process_text(soup.find(id="Property Factors"))

    
    
    utils = safe_search(r'(?<=Utilities)((.|\n)*)(?=Roads)', text)
    if utils:
        utils = re.sub("\xa0", ",", utils)
        utils = re.sub("[^a-zA-Z,]", "", utils)
    
    return {"Utilities": utils}

In [27]:
def get_info(base_url):
    sales_url = get_sales_url(base_url)
    resident_url = get_resident_url(base_url)
    result = {"URL": base_url}
    result.update(extract_parcel(base_url))
    result.update(extract_utilities(base_url))
    result.update(extract_sales_info(sales_url))
    result.update(extract_resident_info(resident_url))
    
    return result

In [28]:
get_info("https://propertyinfo.lyco.org/datalets/datalet.aspx?&UseSearch=no&pin=15-30900210W-000-")

{'Bedrooms': '4',
 'Class': 'R - RESIDENTIAL BUILDING',
 'Full Baths': '1',
 'Half Baths': '0',
 'Land Use Code': '101 - Residential 1 Family',
 'Municipality': '15 - Hepburn Twp',
 'Property Location': '15   FOX ST COGAN STATION',
 'Sales': [('07-28-2005', '1'),
  ('05-31-2005', '98000'),
  ('02-01-1969', '1500')],
 'Square Feet': '4',
 'URL': 'https://propertyinfo.lyco.org/datalets/datalet.aspx?&UseSearch=no&pin=15-30900210W-000-',
 'Utilities': 'Well,Septic',
 'Year Built': '1970'}

In [7]:
import pandas as pd
adf = pd.read_csv("documents/lycoming_centroid.csv")

In [8]:
adf.shape

(51183, 18)

In [29]:
from IPython.display import clear_output
from tqdm import tqdm

columns=["URL", \
         "Bedrooms", \
         "Class", \
         "Full Baths", \
         "Half Baths", \
         "Land Use Code", \
         "Municipality", \
         "Property Location", \
         "Sales", \
         "Square Feet", \
         "Utilities", \
         "Year Built"]

def process(lst):
    df = pd.DataFrame(columns=columns)
    for url in tqdm(lst):
        
        try:
            df = df.append(get_info(url), ignore_index=True)
        except:
            df = df.append(pd.Series(None), ignore_index=True)
        
        clear_output(wait=False)
        
    return df

process([adf["LycoOnline"][1], "asdf"])

100%|██████████| 2/2 [00:03<00:00,  1.82s/it]


,URL,Bedrooms,Class,Full Baths,Half Baths,Land Use Code,Municipality,Property Location,Sales,Square Feet,Utilities,Year Built
0,https://propertyinfo.lyco.org/Datalets/Datalet...,None,V - VACANT LAND > 10 ACRES OR MORE,None,None,113 - Agricultural Vacant Land,41 - Muncy Twp,RT 220 HWY,"[(04-01-2008, 212800), (01-01-1900, 0)]",None,"None,Unknown",None
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
from multiprocessing import Pool
import numpy as np

ranges = [[i[0]-1,i[-1]] for i in np.array_split(np.array(range(1, 51183)), 12)]
ranges[-1][1]+=2

ranges

with Pool(12) as p:
    df_list = p.map(process, [adf["LycoOnline"][i[0]:i[1]] for i in ranges])

100%|██████████| 4265/4265 [3:30:45<00:00,  2.96s/it]


In [35]:
merged_df = pd.concat(df_list)

In [72]:
centroid_df = pd.read_csv("documents/lycoming_centroid.csv", sep=",", header=0)

In [73]:
merged_df = merged_df[merged_df["URL"].notna()].drop_duplicates(subset=["URL"])
centroid_df = centroid_df[centroid_df["LycoOnline"] != ""]

In [74]:
centroid_df =centroid_df.rename(columns={"LycoOnline": "URL"})

In [75]:
centroid_df.shape

(51183, 18)

In [76]:
final_df =merged_df.merge(centroid_df, on="URL", how="inner")

In [84]:
final_df.columns

Index(['URL', 'Bedrooms', 'Class', 'Full Baths', 'Half Baths', 'Land Use Code',
       'Sales', 'Square Feet', 'Utilities', 'Year Built', 'Long', 'Lat',
       'OBJECTID', 'TXID_NMBR', 'PARCEL', 'FULL_NAME', 'OWNER_ADD',
       'OWNER_ADD2', 'OWNER_CITY', 'OWNER_ST', 'OWNER_ZIP', 'MUNIC', 'SITUS',
       'DEED', 'DEED_ACRES', 'GlobalID', 'ORIG_FID'],
      dtype='object')

In [83]:
final_df = final_df.drop(["Property Location", "Municipality"], axis=1)

In [85]:
final_df.to_csv("final_lycoming_data.tsv", sep="\t", header=0, index=False)